# Tennis Demmand - Data Processing

#### Table of Contents
1. [Data Import](#data-import)
2. [Data Transformation and Cleaning](#data-transformation-and-cleaning)
3. [Add Gaussian Noise](#add-gaussian-noise)
4. [Pull multiple pages and consolidate into a single df](#pull-multiple-pages-and-consolidate-into-a-single-df)

#### Data Import

In [30]:
import pandas as pd  
import os  
import json 
from datetime import datetime 
import numpy as np  
import math
import re  
from scipy.stats import norm  
import http.client  

In [3]:
local_modules_path = '../src'
os.sys.path.append(local_modules_path)

In [4]:
from process import *
import api_fetcher

This notebooks was created with the aim of visualizing Tennis demand and create a simple ML model

First, how can we get this data? We are able to pull data from the Amazon's public API via rapidapi. So, we need to search in the product categories, where the Tennis is. It might be likely needed to pull both Women and Men Tenis. Let's take a look at the Women Tenis

In [5]:
amazon_api_client = api_fetcher.ApiClient()
amazon_api_client.http_request(
    method="GET",
    request_url="/products-by-category?category_id=2478868012&page=1&country=US&sort_by=RELEVANCE&product_condition=ALL&is_prime=false&deals_and_discounts=NONE"
)
amazon_api_client.save_response(filename='product_list_us')

In [10]:
with open('../data/api-calls/us_product_list.json', 'r') as file:
    product_list_json = json.load(file)

In [11]:
api_call_response = eval(product_list_json['response'])

In [12]:
api_call_data = api_call_response['data']

In [13]:
pd.DataFrame(api_call_data).sort_values(by="name")

,name,id
12,AWS Courses,courses
1,Alexa Skills,alexa-skills
0,All Departments,aps
2,Amazon Devices,amazon-devices
3,Amazon Explore,live-explorations
4,Amazon Fresh,amazonfresh
5,Amazon Pharmacy,amazon-pharmacy
6,Amazon Warehouse,warehouse-deals
7,Appliances,appliances
8,Apps & Games,mobile-apps


It seems we need to delve depet into to both `fashion-mens` and `fashion-womens`

Let's try out with the path parameter `product-details`

In [23]:
amazon_api_client.http_request(
    method="GET",
    request_url="/search?query=Tenis&country=US&sort_by=RELEVANCE&product_condition=ALL&page=1"
)

{'method': 'GET',
 'request_url': '/search?query=Tenis&country=US&sort_by=RELEVANCE&product_condition=ALL&page=1',
 'response': '{"status":"OK","request_id":"19104c18-c670-47e3-b6d6-90737005fcc8","parameters":{"query":"Tenis","country":"US","sort_by":"RELEVANCE","page":1},"data":{"total_products":43066,"country":"US","domain":"www.amazon.com","products":[{"asin":"B08DQXS11T","product_title":"Tommy Hilfiger Women&#x27;s Lamiss","product_price":"$38.94","product_original_price":"$59.00","currency":"USD","product_star_rating":"4.6","product_num_ratings":2758,"product_url":"https://www.amazon.com/dp/B08DQXS11T","product_photo":"https://m.media-amazon.com/images/I/61XCN70k0UL._AC_UL960_FMwebp_QL65_.jpg","product_num_offers":1,"product_minimum_offer_price":"$38.94","is_best_seller":false,"is_amazon_choice":false,"is_prime":true,"climate_pledge_friendly":false,"sales_volume":"300+ bought in past month","delivery":"FREE delivery Thu, Oct 17 Or fastest delivery Sat, Oct 12","has_variations":tru

In [45]:
tennis_raw_call = amazon_api_client.http_request(
    method="GET",
    request_url="/search?query=Tenis&country=US&sort_by=RELEVANCE&product_condition=ALL&page=1"
)

In [51]:
tennis_raw_call

{'method': 'GET',
 'request_url': '/search?query=Tenis&country=US&sort_by=RELEVANCE&product_condition=ALL&page=1',
 'response': '{"status":"OK","request_id":"a2f663e7-fc60-4b71-b1cb-3d1deb1a4c77","parameters":{"query":"Tenis","country":"US","sort_by":"RELEVANCE","page":1},"data":{"total_products":42714,"country":"US","domain":"www.amazon.com","products":[{"asin":"B08MQP54BM","product_title":"Tommy Hilfiger Men&#x27;s Brecon","product_price":"$50.99","product_original_price":"$75.00","currency":"USD","product_star_rating":"4.7","product_num_ratings":2035,"product_url":"https://www.amazon.com/dp/B08MQP54BM","product_photo":"https://m.media-amazon.com/images/I/71lG78RvwhL._AC_UL960_FMwebp_QL65_.jpg","product_num_offers":1,"product_minimum_offer_price":"$50.99","is_best_seller":false,"is_amazon_choice":false,"is_prime":true,"climate_pledge_friendly":false,"sales_volume":"100+ bought in past month","delivery":"FREE delivery Thu, Oct 17 Or fastest delivery Sun, Oct 13 Prime Try Before You Bu

In [23]:
tennis_raw_result = json.loads(tennis_raw_call["response"])["data"]
tennis_raw_products = tennis_raw_result["products"]

In [43]:
df_tennis_products = pd.DataFrame(tennis_raw_products)

In [24]:
page_1_n_products = pd.DataFrame(tennis_raw_products).shape[0]

In [27]:
total_products = tennis_raw_result["total_products"]

In [28]:
total_products

42688

In [26]:
print(page_1_n_products)

48


In [34]:
total_pages_approx = math.floor(total_products / page_1_n_products)

In [35]:
total_pages_approx

889

In [36]:
tennis_raw_call = amazon_api_client.http_request(
    method="GET",
    request_url="/search?query=Tenis&country=US&sort_by=RELEVANCE&product_condition=ALL&page=800"
)

In [41]:
json.loads(tennis_raw_call["response"])

{'status': 'OK',
 'request_id': 'e7e0a6f4-c61a-497c-81d4-58f7b76b7c02',
 'parameters': {'query': 'Tenis',
  'country': 'US',
  'sort_by': 'RELEVANCE',
  'page': 800},
 'data': {'total_products': 83618,
  'country': 'US',
  'domain': 'www.amazon.com',
  'products': []}}

In [24]:
amazon_api_client.save_response(filename="tenis_products_all")

In [25]:
with open(
    "../data/api-calls/tenis_products_all.json",
    'r'
) as file:
    prod_details_json = json.load(file)

In [26]:
json.loads(prod_details_json['response'])['data']

{'total_products': 43066,
 'country': 'US',
 'domain': 'www.amazon.com',
 'products': [{'asin': 'B08DQXS11T',
   'product_title': 'Tommy Hilfiger Women&#x27;s Lamiss',
   'product_price': '$38.94',
   'product_original_price': '$59.00',
   'currency': 'USD',
   'product_star_rating': '4.6',
   'product_num_ratings': 2758,
   'product_url': 'https://www.amazon.com/dp/B08DQXS11T',
   'product_photo': 'https://m.media-amazon.com/images/I/61XCN70k0UL._AC_UL960_FMwebp_QL65_.jpg',
   'product_num_offers': 1,
   'product_minimum_offer_price': '$38.94',
   'is_best_seller': False,
   'is_amazon_choice': False,
   'is_prime': True,
   'climate_pledge_friendly': False,
   'sales_volume': '300+ bought in past month',
   'delivery': 'FREE delivery Thu, Oct 17 Or fastest delivery Sat, Oct 12',
   'has_variations': True},
  {'asin': 'B08MQP54BM',
   'product_title': 'Tommy Hilfiger Men&#x27;s Brecon',
   'product_price': '$50.99',
   'product_original_price': '$75.00',
   'currency': 'USD',
   'prod

In [27]:
product_data = json.loads(prod_details_json['response'])['data']

In [28]:
tenis_data = pd.DataFrame(product_data['products'])

Once we got a data sample, it is natural to think "what can we model?". As first proposal, me might leverage the `sales_volume` field to forecast demand. As we have lower-bound estimates of the sold figures and this exercise is made exclusively for learning purposes, we could map these estimates into float numbers adding Gaussian Noise.

####  Data Transformation and Cleaning

In [29]:
filtered_columns = [
    'product_price',
    'product_original_price',
    'product_star_rating',
    'product_num_ratings',
    'product_minimum_offer_price',
    'is_prime',
    'climate_pledge_friendly',
    'has_variations',
    'coupon_text',   
    'sales_volume'
]

In [30]:
tenis_data_filtered = tenis_data[filtered_columns]

We have filtered the columns we found useful to estimate `sales_volume`  such as

- `product_price`
- `product_star_ratings`
- `product_num_ratings`

now proceed to clean and transform columns 

In [31]:
tenis_data_processed = tenis_data_filtered.copy()

In [32]:
list(tenis_data_processed['sales_volume'].unique())

['300+ bought in past month',
 '100+ bought in past month',
 '500+ bought in past month',
 'List: ',
 '50+ bought in past month',
 '200+ bought in past month',
 None,
 '400+ bought in past month',
 '800+ bought in past month',
 '900+ bought in past month']

In [33]:
tenis_data_processed[tenis_data_processed['sales_volume']=='List: ']

,product_price,product_original_price,product_star_rating,product_num_ratings,product_minimum_offer_price,is_prime,climate_pledge_friendly,has_variations,coupon_text,sales_volume
3,$41.98,$75.00,4.3,2802,$41.98,True,False,True,NaN,List:
12,$27.95,$60.00,4.3,148,$27.95,False,False,True,NaN,List:
17,$24.95,$50.00,4.2,14449,$24.95,True,False,True,NaN,List:
18,$48.75,$65.00,4.5,904,$48.75,True,False,True,NaN,List:
20,$59.99,$75.00,4.3,63,$59.99,True,False,True,NaN,List:
22,$33.00,$60.00,4,25,$33.00,False,False,True,NaN,List:
23,$57.37,$80.00,4.6,401,$57.37,True,False,True,NaN,List:
24,$44.00,$79.00,4.4,578,$44.00,True,False,True,NaN,List:
27,$52.50,$70.00,4.4,39,$52.50,True,False,True,NaN,List:
41,$44.99,$49.95,4.4,30,$44.99,True,False,True,NaN,List:


In [34]:
tenis_data_processed.head(5)

,product_price,product_original_price,product_star_rating,product_num_ratings,product_minimum_offer_price,is_prime,climate_pledge_friendly,has_variations,coupon_text,sales_volume
0,$38.94,$59.00,4.6,2758,$38.94,True,False,True,NaN,300+ bought in past month
1,$50.99,$75.00,4.7,2033,$50.99,True,False,True,NaN,100+ bought in past month
2,$49.51,$65.00,4.7,16211,$49.51,True,False,True,NaN,500+ bought in past month
3,$41.98,$75.00,4.3,2802,$41.98,True,False,True,NaN,List:
4,$54.69,$70.00,4.6,61020,$54.69,True,False,True,NaN,500+ bought in past month


In [35]:
# Convert to float

tenis_data_processed['product_star_rating'] = tenis_data_processed['product_star_rating'].astype(float);

In [36]:
tenis_data_processed["coupon_discount"] = tenis_data_processed["coupon_text"].map(
    lambda coupon_txt: 
        re.search(pattern="\d{1,2}(\.+\d{1,2})*", string=coupon_txt).group()
        if type(coupon_txt) != float
        else '0.0'
)

# conver to float

tenis_data_processed['coupon_discount'] = tenis_data_processed['coupon_discount'].map(
    lambda discount_str: float(discount_str)
) 

tenis_data_processed.drop(labels=["coupon_text"], axis=1);

In [37]:
# Process categorical data

tenis_data_processed["is_prime"] = pd.get_dummies(
    tenis_data_processed["is_prime"], 
    dtype=float
    )[True]

tenis_data_processed["climate_pledge_friendly"] = pd.get_dummies(
    tenis_data_processed["climate_pledge_friendly"], 
    dtype=float
    )[True]

tenis_data_processed["has_variations"] = pd.get_dummies(
    tenis_data_processed["has_variations"], 
    dtype=float
    )[True]

In [38]:
tenis_data_processed.columns

Index(['product_price', 'product_original_price', 'product_star_rating',
       'product_num_ratings', 'product_minimum_offer_price', 'is_prime',
       'climate_pledge_friendly', 'has_variations', 'coupon_text',
       'sales_volume', 'coupon_discount'],
      dtype='object')

In [39]:
# Reorder columns to leave the predict variable at the end

tenis_data_processed = tenis_data_processed[[
    'product_price',
    'product_original_price',
    'product_star_rating',
    'product_num_ratings',
    'product_minimum_offer_price',
    'is_prime',
    'climate_pledge_friendly',
    'has_variations',
    'coupon_discount',   
    'sales_volume'
]]

#### Add Gaussian Noise

In [40]:
todays_datestr = "2024-10-10"

In [41]:
tenis_data_processed.head(5)

,product_price,product_original_price,product_star_rating,product_num_ratings,product_minimum_offer_price,is_prime,climate_pledge_friendly,has_variations,coupon_discount,sales_volume
0,$38.94,$59.00,4.6,2758,$38.94,1.0,0.0,1.0,0.0,300+ bought in past month
1,$50.99,$75.00,4.7,2033,$50.99,1.0,0.0,1.0,0.0,100+ bought in past month
2,$49.51,$65.00,4.7,16211,$49.51,1.0,0.0,1.0,0.0,500+ bought in past month
3,$41.98,$75.00,4.3,2802,$41.98,1.0,0.0,1.0,0.0,List:
4,$54.69,$70.00,4.6,61020,$54.69,1.0,0.0,1.0,0.0,500+ bought in past month


In [ ]:
tenis_data_processed_test = gaussian_noise(
    df=tenis_data_processed.copy().copy(),
    target_column="sales_volume"
)

In [ ]:
tenis_data_processed_test.head(5)

In [ ]:
tenis_data_processed_test.to_csv(f"../data/processed/tennis_{todays_datestr}.csv", index=False)

In [119]:
tenis_data.to_csv('../data/raw/tenis_products_all_' + todays_datestr + '.csv')

In [100]:
raw_tenis_data = str(product_data)

In [102]:
regexp_search = re.search(pattern="Under", string=raw_tenis_data)

In [104]:
prod_details_df = pd.DataFrame(
    data=product_data
)

In [ ]:
conn = http.client.HTTPSConnection("real-time-amazon-data.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "dfa8842b83msha2bc48dbc5792bdp1cbbd0jsn0bde68569041",
    'x-rapidapi-host': "real-time-amazon-data.p.rapidapi.com"
}

conn.request("GET", "/product-details?asin=B07ZPKBL9V&country=US", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

In [109]:
a = json.loads(data.decode("utf-8"))

#### Pull multiple pages and consolidate into a single df

In [ ]:
for page_num in range(50):
    amazon_api_client.http_request(
        method="GET",
        request_url="/search?query=Tenis&country=US&sort_by=RELEVANCE&product_condition=ALL&page=" + str(page_num)
    )
    amazon_api_client.save_response("tenis_products_" + str(page_num))

In [44]:
with open('../data/api-calls/tenis_products_0.json', 'r') as f:
    d = json.load(f)

In [45]:
with open("../data/api-calls/tenis_products_" + str(8) + ".json", "r") as f:
    a = json.load(f)

In [46]:
json.loads(a["response"])['data']['products']

[]

In [47]:
merged_df = extract_json_df([
    "../data/api-calls/tenis_products_" + str(i) + ".json" 
    for i in range(1, 8)
])

15:21:59.192 | INFO    | Task run 'extract_json_df' - Created task run 'extract_json_df' for task 'extract_json_df'

15:21:59.307 | INFO    | Task run 'extract_json_df' - Finished in state Completed()

In [48]:
merged_df.head()

,asin,product_title,product_price,product_original_price,currency,product_star_rating,product_num_ratings,product_url,product_photo,product_num_offers,product_minimum_offer_price,is_best_seller,is_amazon_choice,is_prime,climate_pledge_friendly,sales_volume,delivery,has_variations,coupon_text
0,B0BHPW3LBX,adidas Men&#x27;s Racer Tr23 Sneaker,$44.52,$80.00,USD,4.6,709,https://www.amazon.com/dp/B0BHPW3LBX,https://m.media-amazon.com/images/I/81jGP9IYOB...,1,$44.52,False,False,True,False,400+ bought in past month,"FREE delivery Tue, Oct 15 Or fastest delivery ...",True,NaN
1,B08MQP54BM,Tommy Hilfiger Men&#x27;s Brecon,$42.50,$75.00,USD,4.7,2028,https://www.amazon.com/dp/B08MQP54BM,https://m.media-amazon.com/images/I/71lG78Rvwh...,1,$42.50,False,False,True,False,50+ bought in past month,"FREE delivery Tue, Oct 15 Or fastest delivery ...",True,NaN
2,B09ZF98TZQ,Reebok Unisex Glide Sneaker,$38.50,$70.00,USD,4.5,502,https://www.amazon.com/dp/B09ZF98TZQ,https://m.media-amazon.com/images/I/71wGqH3EwG...,1,$38.50,False,False,True,False,100+ bought in past month,"FREE delivery Wed, Oct 16 Or fastest delivery ...",True,NaN
3,B08PDXK2P6,Skechers Women&#x27;s Bobs B Cute Shoe,$30.69,$50.00,USD,4.5,40996,https://www.amazon.com/dp/B08PDXK2P6,https://m.media-amazon.com/images/I/71QU68LG-x...,1,$30.69,False,False,True,False,50+ bought in past month,"FREE delivery Sun, Oct 13 on $35 of items ship...",True,NaN
4,B0BZXSLD5H,Under Armour men&#x27;s Charged Commit Trainer...,$60.00,$80.00,USD,4.6,397,https://www.amazon.com/dp/B0BZXSLD5H,https://m.media-amazon.com/images/I/51Ih-2zQ++...,1,$60.00,False,False,True,False,100+ bought in past month,"FREE delivery Wed, Oct 16 Or fastest delivery ...",True,NaN


In [49]:
merged_df = clean_data(df=merged_df.copy())

15:21:59.461 | INFO    | Task run 'clean_data' - Created task run 'clean_data' for task 'clean_data'

/home/santi/current-projects/public-apis/amazon-mlops-dashboard/notebooks/../src/process.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[input_cols] = df[input_cols].fillna(0.0)


15:21:59.536 | INFO    | Task run 'clean_data' - Finished in state Completed()

In [50]:
merged_df = gaussian_noise(df=merged_df.copy(), target_column="sales_volume")

15:21:59.601 | INFO    | Task run 'gaussian_noise' - Created task run 'gaussian_noise' for task 'gaussian_noise'

/home/santi/current-projects/public-apis/amazon-mlops-dashboard/notebooks/../src/process.py:269: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[[target_column_numerical]] = df[[target_column_cleaned]].replace(target_column_vals)


15:21:59.663 | INFO    | Task run 'gaussian_noise' - Finished in state Completed()

In [51]:
merged_df.to_csv("../data/processed/tennis_merged.csv", index=False)